In [ ]:
# This requires some libraries to be installed. These should be at the root level
# of the repository in a requirements.txt file

# assuming you have conda installed, good practice would be to create a virtual environment per repository
# $ conda create -n rd_venv python=3.10
# $ conda activate rd_venv
# this should show the active environment on the prompt
# $ (rd_venv) conda install --file requirements.txt

In [ ]:
from cloudpathlib import AnyPath

In [ ]:
# set a string value for 'where to look for files'
cram_path = 'gs://cpg-perth-neuro-test/cram'

In [ ]:
# create an AnyPath connection to the location 
# https://cloudpathlib.drivendata.org/stable/api-reference/anypath/
# cloudpathlib will see that this is a gs:// path, so AnyPath will really be a CloudPath
# https://cloudpathlib.drivendata.org/stable/api-reference/cloudpath/
cram_path = AnyPath(cram_path)

In [ ]:
# check that the path is assigned accurately? Asserts are either True, or will raise an AssertionError
assert cram_path.exists()

In [ ]:
# list all the files present in that folder
# this produces a generator, not a list (https://realpython.com/introduction-to-python-generators/)
all_files = cram_path.glob('*')
cram_files = cram_path.glob('*.cram')
crai_files = cram_path.glob('*.cram.crai')

In [ ]:
# get all files by making a list from the generator
# this reads every element in the generator and stores it
cram_list = list(cram_files)

# this also 'exhausts' the generator, so it can't be reused
empty_list = list(cram_files)
assert empty_list == []

In [ ]:
# pick an example file from the list
example_file = cram_list[0]

# identify the data type
print(type(example_file))
# <class 'cloudpathlib.gs.gspath.GSPath'>
# API reference: https://cloudpathlib.drivendata.org/stable/api-reference/gspath/
# reference states that a GSPath mimics the interface of Pathlib.Path
# https://docs.python.org/3/library/pathlib.html#pathlib.Path

In [ ]:
# we can use any of these standard Path operations on GSPaths
# printing will usually produce a string representation
print('full path:', example_file)
print('parent folder;', example_file.parent)
print('filename only:', example_file.name)
print('the filename - extension:', example_file.stem)
print('the final suffix:', example_file.suffix)
print('all suffixes:', example_file.suffixes)

# note that a .suffix is a single string, but suffixes will be all suffixes in order
crai_example = AnyPath('gs://bucket/path/filename.cram.crai')
print(crai_example)
print(crai_example.suffix)
print(crai_example.suffixes)

In [ ]:
## Q. how to get all samples with both a cram and a crai file
## A. Sets!
# get all cram & crai files
cram_files = cram_path.glob('*.cram')
crai_files = cram_path.glob('*.cram.crai')

# get all file names using a comprehension (https://www.w3schools.com/python/python_lists_comprehension.asp)
# comprehensions can be sets, lists, dictionaries... very useful
cram_names = {elem.stem for elem in cram_files}

# # this is the equivalent of doing
# cram_names = {}
# for elem in cram_files:
#     cram_names.add(elem.stem)

# .stem only removes one suffix, which is pretty irritating... so our sample.cram and sample.cram.crai 
# files won't automatically match

In [ ]:
# instead, we can do something different... match the partial paths
# take the full name from the cram
cram_files = {elem.name for elem in cram_path.glob('*.cram')}
# trim one suffix from the cram.crai
crai_files = {elem.stem for elem in cram_path.glob('*.cram.crai')}
print(cram_files.intersection(crai_files))

In [ ]:
# this leaves us with the CRAM files that also have a .CRAI in the same folder
# assume the URL we want to embed into is this one:
# http://localhost:60151/load?file=gs://BUCKET/FOLDER/SAMPLE.cram&genome=hg38&name=EXT_SAMPLE
# we can use string formatting for shove our cram files into this String using f-formatting
# https://realpython.com/python-f-strings/
urls = [
    f'http://localhost:60151/load?file={cram_path}/{elem}&genome=hg38' 
    for elem in cram_files.intersection(crai_files)
]
# adding newlines in code is fine, so long as the clause is within brackets to provide context
print(urls)

In [ ]:
# now... need to get the external sample IDs to rename the CRAMs in IGV
# these details will be stored against participants, so we load the Participant Client
from sample_metadata.apis import ParticipantApi
# create a client instance (by default this will know how to find the CPG sample-metadata API)
p = ParticipantApi()

participant_map = p.get_external_participant_id_to_internal_sample_id('perth-neuro')
# >>> [['EXT_1', 'INT_1'], ['EXT_2', 'INT_2'], ...]
# where EXT is the External ID (format will be specific to a project, collaborator, or source)
# the INT ID is always in the format CPG#####
# note that the EXT:INT is not a 1:1 mapping, ext. samples can each be used to generate multiple int. sequences
# we can use a dictionary comprehension to reverse this mapping (https://www.datacamp.com/tutorial/python-dictionary-comprehension)

In [ ]:
# take the list of (external, internal) pairs
# reverse each pair
# make into a dictionary
cpg_to_ext_map = {cpg: ext for (ext, cpg) in participant_map}

In [ ]:
# now we have a way of going from the CPG ID to the external ID, so we can extend the URL string
# http://localhost:60151/load?file=gs://BUCKET/FOLDER/SAMPLE.cram&genome=hg38&name=EXT_SAMPLE
crams_with_indices = cram_files.intersection(crai_files)
urls = [
    f'http://localhost:60151/load?file={cram_path}/{elem}&genome=hg38&name={cpg_to_ext_map[AnyPath(elem).stem]}' 
    for elem in crams_with_indices
]

That looks completely awful, admittedly
it's a few simple steps

`crams_with_indices = cram_files.intersection(crai_files)`
this just checks for intersecting values between two sets, and stores the result as a set
in this case, all samples with both a cram and crai file present.

`urls = [<FORMATTING> for elem in crams_with_indices]`
this scrolls through the intersection-set, and assigns each element in turn to `elem`
`elem` by name is used in the formatting command, so it changes each time

we know that each element of the intersection will be a CRAM file (CPG####.cram), so
`AnyPath(elem).stem` just loads that string as an AnyPath instance, and `.stem`
removes the extension. This is overkill, there are much simpler ways to do that part...

`cpg_to_ext_map[AnyPath(elem).stem]` takes the CPG#### ID, and uses it as a key
in the Internal -> External dict, pulling out the external ID as a String

I don't know about you, but when I ran this, there was an exception looking up an external ID
that was because one sample was present on file but not in the sample_metadata API :thinking_face:
we could pre-filter the dictionary, and skip samples where we don't have the result...

OR we can use this to learn about Exceptions in Python! (yaaay)
https://docs.python.org/3/tutorial/errors.html

In [ ]:
# get the intersecting samples
crams_with_indices = cram_files.intersection(crai_files)

# create this string once, without f-formatting
format_string = 'http://localhost:60151/load?file=gs://cpg-perth-neuro-test/cram/{internal}&genome=hg38&name={external}' 

# create an empty list
urls = []

# and keep track of these
problems = []

# loop over all the cram files
for cram_name in crams_with_indices:
    # indent this whole section inside a `try` so Python knows we might have to handle problems
    try:
        # split on '.', and take the first part. CPG###.cram -> CPG###
        cpg_id = cram_name.split('.')[0]
        ext_id = cpg_to_ext_map[cpg_id]
        
        # you .append to lists, and .add to sets
        # tell the formatter which variables to put where
        urls.append(format_string.format(internal=cpg_id, external=ext_id))
    
    # failing to find a dictionary key will be a KeyError
    # this except can be anywhere after the problem
    # i.e. it won't fail to catch the problem unless the next line is an `except`
    except KeyError as ke:
        # print an explanatory message, but let the code continue
        print(f'There was a problem looking up Ext. ID for sample {cpg_id}')
        problems.append(cpg_id)

In [ ]:
for url in urls:
    print(url)

In [ ]:
# mention these to someone...
print(problems)

In [ ]:
# and then we want to write these to a file
# by default python doesn't add line separators between the things we print
# we could add `\n` to each print, or let the OS determine what the correct newline is for the platform
from os import linesep
with open('output_file.txt', 'w') as handle:
    for url in urls:
        handle.write(url + linesep)